In [0]:
import requests
from IPython.core.display import HTML
HTML(f"""
<style>
@import "https://cdn.jsdelivr.net/npm/bulma@0.9.4/css/bulma.min.css";
</style>
""")

# Filtering
Filters are arbitrary linear functions and can therefore be used to perform many different kinds of operations. In this exercise, we will explore image filtering, a powerful technique in image processing, to understand how filters can be applied to modify, or extract specific features from images.
As a first step, import the necessary libraries:


In [0]:
import numpy as np
from skimage import data, color
from scipy.ndimage import convolve, correlate
from scipy import stats, signal
from skimage import filters, feature, transform, io
from skimage.transform import pyramid_gaussian
import matplotlib.pyplot as plt

We will use the same set of example images from the Scikit-Image library that were used in the tutorial. You can use and modify the code below to load and inspect any of the images: 


In [0]:
print(data.__all__)
plt.imshow(data.coins(), cmap="gray")
plt.axis(False)


---
**Task 1 (easy): Shifting pixels👩‍💻♾️**
Construct a filter that shifts pixels in the image by $n$ pixels to the right.
1. Create a function with parameter `n`
 that returns a kernel that can be used to shift an image `n`
 pixels to the right.
2. Apply the filter to one of the example images and show the results. Try out multiple different values for `n`
.
3. Make the functions `shift_to_left`
, `shift_to_up`
, and `shift_to_down`
 to shift the input images $n$ pixels to the left, up and down. Use one of the example images to test these functions and evaluate if the images are correctly shifted. 
4. (Optional) Modify the filter to not only shift the pixel to the right (or the other directions) but also decrease its brightness.


---

In [0]:
def shift_to_right(n):
    """
    Param n: number of pixels to shift to the right.

    Returns: kernel for shifting.
    """
    # write your solution here
    ...


---
**Task 2 (medium): Gaussian blurring👩‍💻**
Construct a Gaussian filter and apply it to an image by following the steps below:
1. Design a 3x3 kernel (filter) for Gaussian blurring. Each element of the filter should be filled with values that approximate a Gaussian distribution.


**Hint**
The filter was covered during the lecture.

2. Plot the Gaussian Blur filter.
3. Apply the filter to one of the example image and display the original and blurred images side by side.
4. Explain the underlying reasons for the observed effects produced by this filter.
5. Implement the increased Gaussian blurring kernel shown below. Apply the filter to the selected image and compare the results to the results obtained in the previous steps. Explain the observed effects. 


$$
\begin{equation}
\frac{1}{256}
\begin{bmatrix}
1 & 4 & 6 & 4 & 1 \\
4 & 16 & 24 & 16 & 4 \\
6 & 24 & 36 & 24 & 6 \\
4 & 16 & 24 & 16 & 4 \\
1 & 4 & 6 & 4 & 1 \\
\end{bmatrix}
\end{equation}
$$

---

In [0]:
# write your solution here


---
**Task 3 (medium): Template matching👩‍💻**
In the following exercise you will are going to implement template matching on the sunflower image (`sunflowers.jpg`
) found in the data library for this week's exercises.
1. Load the image, convert it to grayscale and normalize its pixel values. 
2. Define a rectangular region of interest (ROI) within the image. This ROI will serve as the template you want to extract from the image. and it should include the flower head of a single sunflower. Extract the template and plot it.
3. Implement template matching by using the [`skimage.feature.match_template`
](https://scikit-image.org/docs/stable/auto_examples/features_detection/plot_template.html)
 function. The function returns a two-dimensional array (correlation map) that indicate the similarity or correlation between the template and regions of the input image. Plot the results.
4. Was the template correctly matched to the original image? Were there any other areas that appeared to be potential matches?


---

In [0]:
# write your solution here


---
**Task 4 (medium): Sharpening👩‍💻💡**
In this task, you will create filters to sharpen the pixels of an input image, making details look clearer than in the original image. Here are two filters used for image sharpening:

$$
K_1=\frac{1}{9}\left[
	\begin{array}{r r r}
		-1 & -1 & -1 \\
		-1 & 17 & -1 \\
		-1 & -1 & -1
	\end{array}
\right],
K_2=\left[
	\begin{array}{r r r}
		-1 & -1 & -1 \\
		-1 &  9 & -1 \\
		-1 & -1 & -1
	\end{array}
\right]
$$
Notice that in this case the filter values sum to one.
1. Why do sharpen filters contain a single positive value at their centers, which are completely surrounded by negative values?

2. Define the above filter kernels as Numpy arrays.

3. Use the filter kernels $K_1$ and $K_2$ to sharpen one of the example images from the Scikit-Image library. Show the two filtered images and compare them with the original input image.

4. Implement the sharpening filter shown below and apply it to the selected image. Reflect on the effects of increasing the size of the sharpening filter. 



$$
\begin{equation}
\frac{1}{8}
\begin{bmatrix}
-1 & -1 & -1 & -1 & -1 \\
-1 & 2 & 2 & 2 & -1 \\
-1 & 2 & 8 & 2 & -1 \\
-1 & 2 & 2 & 2 & -1 \\
-1 & -1 & -1 & -1 & -1 \\
\end{bmatrix}
\end{equation}
$$

---

In [0]:
# write your solution here